# Visual Auditor Package Demo (German Credit Dataset)
Author: David Munechika (david.munechika@gatech.edu)

### Install Packages and Libraries

In [1]:
# Import packages
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from visual_auditor import SliceFinder
import json

# Import visual auditor module
import visual_auditor

# Expand Jupyter notebook width
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### Data Preprocessing and Model Training

In [2]:
# Helper function for binning numerical features
def bin_feature(feature):
    bins = np.histogram_bin_edges(german_credit_data[feature], bins='auto', range=None, weights=None)
    german_credit_data[feature] = pd.cut(german_credit_data[feature], bins, labels=[x for x in range(len(bins) - 1)], right=True, include_lowest=True, duplicates='drop')
    intervals = []
    for i in range(len(bins) - 1):
        intervals.append(f' {int(bins[i])} - {int(bins[i+1])}')
    return intervals

# Helper function for mapping encodings to original values
def map_encodings(filename, features, encodings):
    slices_file = open(filename, "r")
    data = json.dumps(json.loads(slices_file.read()))
    slices_file.close()
    for feature in features:
        for i in range(len(encodings[feature])):
            index = len(encodings[feature]) - i - 1
            data = data.replace(f'{feature}: {index}', f'{feature}: {encodings[feature][index]}')
    with open(filename, 'w') as outfile:
        json.dump(json.JSONDecoder().decode(data), outfile)

In [3]:
# Load German Credit dataset
german_credit_data = pd.read_csv(
    "data/german.csv",
    names=[
        "Checking Account", "Duration", "Credit History", "Purpose", "Credit Amount", "Savings Account",
        "Employment", "Installment Rate", "Relationship/Sex", "Debtors/Guarantors", "Residence Since", "Property", "Age", "Installment Plans", "Housing", "Existing Credits", "Job", "Maintenance", "Telephone", "Foreign", "Risk"],
        engine='python',
        na_values="NA")

# Drop NA values
german_credit_data = german_credit_data.dropna()

# Bin numerical features
encoders = {}
encodings = {}
numerical_features = ["Age", "Credit Amount", "Duration"]
for feature in numerical_features:
    encodings[feature] = bin_feature(feature)

# Encode categorical features
for column in german_credit_data.columns.difference(numerical_features):
    if german_credit_data.dtypes[column] == np.object:
        le = LabelEncoder()
        german_credit_data[column] = le.fit_transform(german_credit_data[column])
        encoders[column] = le
        encodings[column] = le.classes_
        print(column, le.classes_, le.transform(le.classes_))
encodings["Checking Account"] = [' 0 - 200', '< 0', '> 200', 'No checking account']
encodings["Savings Account"] = [' 100 - 500', ' 500 - 1000', '< 100', '> 1000', 'Unknown']

# Separate Target values
X, y = german_credit_data[german_credit_data.columns.difference(["Risk"])], german_credit_data["Risk"]

# Train a classifier
rfc = RandomForestClassifier(max_depth=5, n_estimators=10)
rfc.fit(X, y)

Checking Account ['0 <= ... < 200 DM' '< 0 DM' '>= 200 DM' 'No checking account'] [0 1 2 3]
Credit History ['All credits paid back' 'Critical account' 'Delay in paying'
 'Existing credits paid back' 'No credits taken'] [0 1 2 3 4]
Debtors/Guarantors ['Co-applicant' 'Guarantor' 'None'] [0 1 2]
Employment ['1 <= ... < 4 years' '4 <= ... < 7 years' '< 1 year' '>= 7 years'
 'Unemployed'] [0 1 2 3 4]
Foreign ['No' 'Yes'] [0 1]
Housing ['For free' 'Own' 'Rent'] [0 1 2]
Installment Plans ['Bank' 'None' 'Stores'] [0 1 2]
Job ['Management' 'Skilled' 'Unemployed' 'Unskilled'] [0 1 2 3]
Property ['Car or other' 'Life insurance' 'Real estate' 'Unknown or none'] [0 1 2 3]
Purpose ['Appliances' 'Business' 'Car (new)' 'Car (used)' 'Education' 'Furniture'
 'Others' 'Radio/TV' 'Repairs' 'Retraining'] [0 1 2 3 4 5 6 7 8 9]
Relationship/Sex ['Female (separated/married)' 'Male (married)' 'Male (separated)'
 'Male (single)'] [0 1 2 3]
Savings Account ['100 <= ... < 500 DM' '500 <= ... < 1000 DM' '< 100 DM'

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

### Precompute Data for Demo (Internal Use)

In [4]:
# Map encodings
features = ["Age", "Credit Amount", "Duration", "Checking Account", "Credit History", "Debtors/Guarantors", "Employment", "Foreign", "Housing", "Installment Plans", "Job", "Property", "Purpose", "Relationship/Sex", "Savings Account", "Telephone"]
map_encodings('gc_slices.json', features, encodings)
map_encodings('gc_common_samples.json', features, encodings)
map_encodings('gc_overlapping_samples.json', features, encodings)
map_encodings('gc_reverse_slices.json', features, encodings)
map_encodings('gc_reverse_common_samples.json', features, encodings)
map_encodings('gc_reverse_overlapping_samples.json', features, encodings)

### Interact with Visual Auditor

In [5]:
# Generate the visual auditor for this model and dataset
visual_auditor.find_slices_and_visualize(rfc, (X, y), k=100, epsilon=0.01, degree=2, max_workers=4, precompute=True, prefix='gc_')

<iframe
 srcdoc="<!doctype html><html lang="en"><head><meta charset="utf-8"/><link rel="icon" href="/favicon.ico"/><meta name="viewport" content="width=device-width,initial-scale=1"/><meta name="theme-color" content="#000000"/><meta name="description" content="Interactive Scalable Auditing of Model Biases and Vulnerabilities with Interpretable Mitigation"/><link rel="apple-touch-icon" href="/logo192.png"/><link rel="manifest" href="/manifest.json"/><title>Visual Auditor</title><style>body{margin:0;font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,Oxygen,Ubuntu,Cantarell,"Fira Sans","Droid Sans","Helvetica Neue",sans-serif;-webkit-font-smoothing:antialiased;-moz-osx-font-smoothing:grayscale;background:#fff}code{font-family:source-code-pro,Menlo,Monaco,Consolas,"Courier New",monospace}*{scrollbar-width:thin;scrollbar-color:hsla(0,0%,60.8%,.5) transparent}::-webkit-scrollbar{width:5px}::-webkit-scrollbar-track{background:0 0}::-webkit-scrollbar-thumb{background-color:hsla(0,0%,60.8%,.5);border-radius:20px;border:transparent}.nav-container{text-align:center;z-index:2}.settings{margin:1rem}.settings:hover{opacity:1%;cursor:pointer}.left-container{text-align:center;padding:.5rem}.MuiFormControlLabel-label{color:#5f5f5f;font-weight:700}h1{color:#5f5f5f}h2{font-size:1.25rem;padding:.5rem}h2,p{color:#5f5f5f;font-weight:300}p{text-align:left;font-size:1.1rem;padding:0 1rem}.similar-slices-list{font-size:1rem;color:#5f5f5f;font-weight:300;text-align:left}.similar-slice{padding:.25rem 0}.thin{font-weight:300}.right-container{text-align:center;padding:.5rem}.main-container{padding:1rem;text-align:left;overflow-x:scroll;background-color:#fff}img{margin-left:10rem;margin-right:10rem;width:60rem}.hull{fill:rgba(184,125,0,.705);stroke:rgba(184,125,0,.705);opacity:.5;stroke-width:32px;stroke-linejoin:round}.link{stroke:#000;stroke-width:1.5px}.node{cursor:move;fill:#ccc;stroke-width:1.5px}.node.fixed{fill:red}</style></head><body><noscript>You need to enable JavaScript to run this app.</noscript><div id="root"></div><script>!function(e){function r(r){for(var n,i,l=r[0],a=r[1],f=r[2],c=0,s=[];c<l.length;c++)i=l[c],Object.prototype.hasOwnProperty.call(o,i)&&o[i]&&s.push(o[i][0]),o[i]=0;for(n in a)Object.prototype.hasOwnProperty.call(a,n)&&(e[n]=a[n]);for(p&&p(r);s.length;)s.shift()();return u.push.apply(u,f||[]),t()}function t(){for(var e,r=0;r<u.length;r++){for(var t=u[r],n=!0,l=1;l<t.length;l++){var a=t[l];0!==o[a]&&(n=!1)}n&&(u.splice(r--,1),e=i(i.s=t[0]))}return e}var n={},o={1:0},u=[];function i(r){if(n[r])return n[r].exports;var t=n[r]={i:r,l:!1,exports:{}};return e[r].call(t.exports,t,t.exports,i),t.l=!0,t.exports}i.m=e,i.c=n,i.d=function(e,r,t){i.o(e,r)||Object.defineProperty(e,r,{enumerable:!0,get:t})},i.r=function(e){"undefined"!=typeof Symbol&&Symbol.toStringTag&&Object.defineProperty(e,Symbol.toStringTag,{value:"Module"}),Object.defineProperty(e,"__esModule",{value:!0})},i.t=function(e,r){if(1&r&&(e=i(e)),8&r)return e;if(4&r&&"object"==typeof e&&e&&e.__esModule)return e;var t=Object.create(null);if(i.r(t),Object.defineProperty(t,"default",{enumerable:!0,value:e}),2&r&&"string"!=typeof e)for(var n in e)i.d(t,n,function(r){return e[r]}.bind(null,n));return t},i.n=function(e){var r=e&&e.__esModule?function(){return e.default}:function(){return e};return i.d(r,"a",r),r},i.o=function(e,r){return Object.prototype.hasOwnProperty.call(e,r)},i.p="/";var l=this["webpackJsonpvisual-auditor"]=this["webpackJsonpvisual-auditor"]||[],a=l.push.bind(l);l.push=r,l=l.slice();for(var f=0;f<l.length;f++)r(l[f]);var p=a;t()}([])</script><script type="text/javascript">/*! For license information please see 2.d769a9af.chunk.js.LICENSE.txt */
(this["webpackJsonpvisual-auditor"]=this["webpackJsonpvisual-auditor"]||[]).push([[2],[function(e,t,n){"use strict";e.exports=n(234)},function(e,t,n){"use strict";e.exports=n(240)},function(e,t,n){"use strict";function r(){return(r=Object.assign||function(e){for(var t=1;t<arguments.length;t++){var n=argument